<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week4/Face_Recognition_v3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition

이번 과제에서는, 얼굴 인식 시스템을 개발합니다. 이번 과제에서 선보일 많은 아이디어는 [FaceNet](https://arxiv.org/pdf/1503.03832.pdf) 에서 차용했습니다. 추가적으로 강의에서 [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf)에 대해서도 이야기 했던 바 있습니다.

얼굴 인식 문제는 일반적으로 두 가지 카테고리로 분류할 수 있습니다.

- **Face Verification** - "이 사람이 유효한 사람입니까?" 라는 질문. 예를 들어 일부 공항에서는 시스템이 여권을 스캔하도록 한 다음 실제 여권 소지자인지 얼굴을 인식하여 세관을 통과 할 수 있도록 합니다. 얼굴 인식 잠금 해제를 지원하는 스마트폰도 얼굴 인증을 사용하고 있습니다. 이는 1:1 매칭 문제입니다.
- **Face Recognition** - "이 사람은 누구입니까?" 라는 질문. 예를 들어 지난 강의에서 다른 어떤 신원 확인 없이 사무실에 들어오는 Baidu 직원들의 [얼굴 인식 동영상](https://www.youtube.com/watch?v=wr4rx0Spihs)을 보여주었습니다. 이는 1 : K 매칭 문제입니다.

FaceNet은 얼굴 이미지를 128 개의 숫자로 구성된 벡터로 인코딩하는 신경망을 학습합니다. 그런 다음 두 벡터를 비교하여 두 사진이 같은 사람인지 확인할 수 있습니다.

**이 과제에서 다음을 수행합니다.**
- Triplet Loss 기능 구현
- 사전 훈련 된 모델을 사용하여 얼굴 이미지를 128 차원 인코딩으로 매핑
- 이 인코딩을 사용하여 얼굴 확인 및 얼굴 인식을 수행합니다.

#### Channels-first notation

- 이 과제에서는 강의 및 지난 프로그래밍 과제에 사용 된 "channels last" 규칙과 달리 **"channels first"** 규칙을 사용하여 ConvNet 활성화를 나타내는 사전 학습 된 모델을 사용합니다.
- 즉, 이미지 배치는 $(m, n_H, n_W, n_C)$ 대신 $(m, n_C, n_H, n_W)$ 형태의 shape가 됩니다.
- 이 두 가지 규칙 모두 오픈 소스 구현 사이에서 상당한 견인력을 가지고 있습니다. 딥 러닝 커뮤니티에는 아직 통일 된 표준이 없습니다.

#### Load packages

필요한 패키지를 받아봅시다.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

## 0 - Naive Face Verification

Face verification 에서는 두 개의 이미지가 제공되며 두 이미지가 같은 사람인지 확인해야 합니다. 이를 수행하는 가장 간단한 방법은 두 이미지를 픽셀 단위로 비교하는 것입니다. Raw 이미지 사이의 거리가 선택한 임계 값보다 작으면 같은 사람일 가능성이 높은 것입니다.

<img src="arts/pixel_comparison.png" style="width:380px;height:150px;">
<center>그림 1</center>

- 물론, 이 알고리즘은 조명의 변화, 사람 얼굴의 방향, 심지어 머리 위치의 사소한 변화 등으로 인해 픽셀 값이 극적으로 변하기 때문에 성능이 매우 떨어집니다.
- Raw 이미지를 사용하는 대신 $f(img)$ 인코딩을 배울 수 있음을 알 수 있습니다.
- 각 이미지에 인코딩을 사용하여 요소 별 비교를 통해 두 사진이 같은 사람인지 여부를보다 정확하게 판단 할 수 있습니다.

## 1 - Encoding face images into 128-dimensional vector

### 1.1 - Using a ConvNet to compute encodings

FaceNet 모델은 훈련하는 데 많은 데이터와 오랜 시간이 걸립니다. 따라서 다른 딥 러닝 애플리케이션의 일반적인 관행에 따라 다른 사람이 이미 훈련 한 가중치를 불러와 보겠습니다. 네트워크 아키텍처는 [Szegedy * et al. *](https://arxiv.org/abs/1409.4842)의 Inception 모델을 따릅니다. 이번 과제에서는 초기 네트워크 구현을 제공하고 있습니다. `inception_blocks_v2.py` 파일에서 구현 방법을 볼 수 있습니다 (Jupyter 노트북 상단의 "파일 -> 열기 ..."로 이동하면 '.py' 파일을 포함하고 있는 파일 디렉토리를 열 수 있습니다).

알아야 할 핵심 사항은 다음과 같습니다.

- 이 네트워크는 96x96 차원의 RGB 이미지를 입력으로 사용합니다. 특히, 얼굴 이미지 (또는 $m$개의 얼굴 이미지 배치)를 $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ shape의 텐서로 입력합니다.
- 각 입력 얼굴 이미지를 128 차원 벡터로 인코딩하는 $(m, 128)$ 모양의 행렬을 출력합니다.

아래 셀을 실행하여 얼굴 이미지 모델을 만듭니다.

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [ ]:
print("Total Params:", FRmodel.count_params())

**모범 답안**
<table>
<center>
Total Params: 3743280
</center>
</table>

128개의 fully-connected layer를 최종 레이어로 사용함으로서, 출력이 128개 크기의 인코딩 벡터인지 확인합니다. 그런 다음 인코딩을 사용하여 다음과 같이 두 얼굴 이미지를 비교합니다.

<img src="arts/distance_kiank.png" style="width:680px;height:250px;">

<center>그림 2 : 두 인코딩과 임계 값 사이의 거리를 계산하여 두 사진이 동일한 사람을 나타내는 지 확인할 수 있습니다/</center>

따라서 다음과 같은 경우 인코딩이 좋습니다.
- 같은 사람의 두 이미지 인코딩이 서로 매우 유사합니다.
- 사람이 다른 두 이미지의 인코딩은 매우 다릅니다.

삼중 항 손실 함수(Triplet loss function)는 이를 형식화하고 동일한 사람(Anchor 및 Positive)의 두 이미지 인코딩을 더 가깝게 끌어당기고, 다른 사람 (Anchor, Negative)의 두 이미지 인코딩을 더 멀리 밀어내려고 합니다.

<img src="arts/triplet_comparison.png" style="width:280px;height:150px;">

<center>그림 3 : 다음 파트에서, 왼쪽에서 오른쪽으로 그림을 호출합니다 : Anchor(A), Positive(P), Negative(N)</center>

### 1.2 - The Triplet Loss

특정 이미지 $x$에 대해, 인코딩 $f(x)$를 인공 신경망에 의해서 계산되는 함수로 정의하겠습니다.

<img src="images/f_x.png" style="width:380px;height:150px;">

훈련 과정에서, 우리는 $(A, P, N)$ 이미지에 대한 삼중항(triplet)을 사용합니다.
- A는 "Anchor" 이미지로, 기준이 되는 사람의 이미지 입니다.
- P는 "Positive" 이미지로, Anchor 이미지와 동일한 사람의 이미지 입니다. 
- N은 "Negative" 이미지로, Anchor 이미지와 다른 사람의 이미지 입니다.

위 삼중항은 훈련 데이터셋에서 선별된 이미지로 구성됩니다. 따라서 우리는 $(A^{(i)}, P^{(i)}, N^{(i)})$ 를 $i$번째 훈련 데이터의 삼중항으로 칭하겠습니다.

여러분은 특정 사람의 이미지 $A^{(i)}$가 Negative 이미지 $N^{(i)}$ 보다 Positive 이미지 $P^{(i)}$ 에 더 가깝도록하고 싶습니다. 이 때 둘 사이의 최솟값은 적어도 $\alpha$보다 커야 합니다.

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

따라서 다음 "Triplet Loss"를 최소화 하려고 합니다.

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

이번 과제에선 "$[z]_+$"표기법을 사용하여 $max(z, 0)$ 를 나타냅니다.

Notes:
- term (1)는 주어진 삼중 항에 대한 Anchor "A"와 Positive "P"사이의 제곱 거리입니다. 당신은 이것이 작기를 원합니다.
- term (2)는 주어진 삼중 항에 대한 Anchor "A"와 Negative "N"사이의 제곱 거리입니다. 이 값은 상대적으로 커야합니다. 음수를 어떤 변수를 최소화하는 것은 해당 변수를 최대화하는 것과 같기 때문에 term 앞에 음수 기호가 있습니다.
- $\alpha$를 마진 이라고 합니다. 직접 선택할 수 있는 하이퍼 파라미터입니다. 이번 과제에선 $\alpha = 0.2$를 사용합니다.

대부분의 구현은 L2 정규화가 1이되도록 인코딩 벡터의 크기를 조정합니다 (즉, $\mid\mid f(img) \mid\mid_2$ = 1); 따라서 이 과제에서 L2 정규화에 대해서 크게 걱정할 필요가 없습니다.

**연습 문제** : (3)번 공식에 정의된 대로, 삼중항 손실(triplet loss)를 계산하는 함수를 구현하세요. 아래 네 단계를 따라서 구현합니다.

1.  "anchor"와 "positive" 이미지 사이의 거리를 계산하세요 : $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. "anchor"와 "negative" 이미지 사이의 거리를 계산하세요 : $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. 각 훈련 데이터마다 위의 공식을 계산합니다 : $ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
4. 0으로 최대 값을 취하고 훈련 예제를 모두 더하여 전체 공식을 계산합니다 : 
  $$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2+ \alpha \large ] \small_+ \tag{3}$$

#### Hints

- 유용한 함수들 : `tf.reduce_sum(), tf.square(), tf.subtract(), tf.add(), tf.maximum()`
- 1번과 2번 단계에 대해서, $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$와 $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$의 각 entry를 더합니다.
- 4번 단계에선 모든 훈련 데이터의 결과를 더합니다.

#### Additional Hints

- L2 정규화의 제곱은 제곱 오차의 합이라는 것을 기억하세요 :  $||x - y||_{2}^{2} = \sum_{i=1}^{N}(x_{i} - y_{i})^{2}$
- `anchor`, `positive`, `negative` 인코딩은 `(m, 128)`의 shape를 가지고 있습니다. 여기서 m은 훈련 데이터의 갯수이고, 128은 단일 예제를 인코딩하는데 사용되는 element의 갯수입니다.
- 1 단계와 2 단계에서는 학습 예제의 수 `m`을 유지하고 각 인코딩의 128 개 값을 합산합니다.
[tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/math/reduce_sum) 함수에는 `axis` 매개 변수가 있습니다. 이 변수는 합계가 적용되는 축을 설정하는데 사용됩니다.
- 텐서에서 마지막 축을 선택하는 한 가지 방법은 음의 인덱싱 (`axis = -1`)을 사용하는 것입니다.
- 4 단계에서 학습 예제를 합산하면 결과는 단일 스칼라 값이됩니다.
- `tf.reduce_sum`이 모든 축을 합산도록 하려면 기본값 `axis = None`을 유지해야 합니다.

In [ ]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = None
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = None
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = None
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = None
    ### END CODE HERE ###
    
    return loss

In [ ]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>loss</b>
        </td>
        <td>
           528.143
        </td>
    </tr>

</table>

## 2 - Loading the pre-trained model

FaceNet은 삼중 항 손실을 최소화하는 방법으로 훈련됩니다. 하지만 훈련에는 많은 데이터와 많은 계산이 필요하기 때문에 당장 처음부터 훈련하지는 않을 것입니다. 대신 이전에 훈련 된 모델을 불러옵니다. 다음 셀을 사용하여 사전 훈련된 모델을 불러와보세요. 실행하는 데 몇 분 정도 걸릴 수 있습니다.

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

다음은 세 가지 인물 사진 간의 인코딩 간의 거리에 대한 몇 가지 예입니다.

<img src="arts/distance_matrix.png" style="width:380px;height:200px;">
<center>그림 4 : 세 가지 인물 사진 간의 거리 출력 예</center>

이제 이 모델을 사용하여 Face Verification및 Face Recognition을 수행해 보겠습니다!

## 3 - Applying the model

여러분은 현재 허가된 직원만이 건물에 들어갈 수 있도록 얼굴 인식을 지원하는 시스템을 구축하고 있습니다.

그 건물에 살고 있거나, 건물 내부에서 일하는 사람들의 목록에 접근하여 그 사람임을 인식할 수 있는 **Face verification** 시스템을 구축하려고합니다. 건물에 입장하려면 각 사람이 입구에서 신분증을  찍어 신원을 확인해야합니다. 그런 다음 얼굴 인식 시스템은 그 사람이 신분증에 나타난 사람과 동일한 사람인지 확인합니다.

### 3.1 - Face Verification

건물에 들어올 수 있는 각 사람에 대해 하나의 인코딩 벡터를 포함하는 데이터베이스를 구축해 보겠습니다. 인코딩을 생성하기 위해 지정된 이미지에서 모델의 forward propagation을 실행하는 `img_to_encoding(image_path, model)`을 사용합니다.

다음 코드를 실행하여 데이터베이스를 빌드합니다(python 딕셔너리로 표시됨). 이 데이터베이스는 각 사람의 이름을 얼굴의 128 차원 인코딩에 매핑합니다.

In [ ]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

이제 누군가가 건물 현관에 나타나서 자신의 ID 카드를 스캔하면(이 때 이름을 알려줍니다), 데이터베이스에서 인코딩을 찾아서 현관에 서있는 사람이 ID 카드의 정보와 일치하는지 확인하는 데 사용할 수 있습니다.

**연습 문제** : 건물 현관 프론트에 비치된 카메라로 찍은 상대방의 사진(`image_path`이 실제로 ID 카드의 사진과 일지하는지 확인하는 `verify()` 함수를 구현하세요. 다음 단계를 거쳐서 구현합니다.

1. `image_path`의 이미지 인코딩을 계산합니다.
2. 이 인코딩과 데이터베이스에 저장된 ID 카드의 이미지 인코딩 사이의 distance를 계산하세요.
3. distance가 0.7 미만이면 문을 열어 출입을 허용하고, 그렇지 않으면 출입을 거부합니다.

- 위에 제시된대로 L2 정규화된 distance [np.linalg.norm](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html)를 사용해야합니다.
- (참고 : 이 구현에서는 L2 distance의 제곱이 아닌 그냥 L2 distance를 임계 값 0.7과 비교하십시오.)

#### Hints

- `identity`는 `database` 딕셔너리의 키이기도 한 문자열입니다.
- `img_to_encoding`에는 `image_path`와 `model`의 두 가지 매개 변수가 있습니다.

In [ ]:
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = None
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = None
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if None:
        print("It's " + str(identity) + ", welcome in!")
        door_open = None
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = None
        
    ### END CODE HERE ###
        
    return dist, door_open

Younes는 사무실에 들어 가려고 하고 카메라가 그의 사진을 찍었습니다 ("arts/camera_0.jpg"). 이 그림에서 Face verification 알고리즘을 실행 해 보겠습니다.

<img src="arts/camera_0.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

**모범 답안**:

<table>
    <tr>
        <td>
            <b>it's younes, welcome in!</b>
        </td>
        <td>
           (0.65939283, True)
        </td>
    </tr>

</table>

사무실에서 일하지 않는 베누아는 키안의 신분증을 훔쳐 사무실에 들어 가려고 합니다. 카메라는 Benoit("arts/camera_2.jpg)의 사진을 찍었습니다. Face verification 알고리즘을 실행하여 benoit가 입장 할 수 있는지 확인해 봅시다.

<img src="arts/camera_2.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

**모범 답안**:

<table>
    <tr>
        <td>
            <b>It's not kian, please go away</b>
        </td>
        <td>
           (0.86224014, False)
        </td>
    </tr>

</table>

### 3.2 - Face Recognition

앞서 만든 Face Verification 시스템이 잘 작동하고 있는 것 같습니다. 하지만, 위 사례에서 Kian이 신분증을 도난당햇기 때문에 다음날 사무실에 왔을 때 들어갈 수가 없는 문제가 생깁니다.

이를 해결하기 위해 Face Verification 시스템을 Face Recognition 시스템으로 변경하고 싶습니다. 이렇게 하면 더이상 ID 카드를 휴대할 필요가 없습니다. 승인된 사람이 건물까지 걸어가면 문이 열립니다.

이미지를 입려갛여 승인된 사람 중 한 명에 속하는지(속한다면 누구인지) 파악하는 Face Recognition 시스템을 구현합니다. 이전 Face verification 시스템과 달리 더이상 사전 입력으로 사람의 이름을 받지 않습니다.

**연습 문제** : `who_is_it()` 함수를 구현합니다. 다음 단계를 거쳐야 합니다.

1. `image_path` 이미지의 대상 인코딩을 계산합니다.
2. 데이터베이스에서 대상 인코딩과의 거리가 가장 짧은 인코딩을 가진 이미지를 찾습니다.
  - `min_dist` 변수를 충분히 큰 수(100)로 초기화합니다. 입력값의 인코딩에 가장 가까운 인코딩이 무엇인지 추적하는데 도움이 됩니다.
  - 데이터베이스 딕셔너리의 이름과 인코딩에 대해서 아래 작업을 반복합니다. `for (name, db_enc) in database.items()` 코드를 수행하세요.
    - 대상 인코딩과 데이터베이스에서 현재 인코딩 사이의 L2 distance를 계산합니다.
    - 이 거리가 min_dist보다 작 으면 `min_dist`를 `dist`로, `identity`를 `name`으로 설정합니다.

In [ ]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = None
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in None:
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = None

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if None:
            min_dist = None
            identity = None

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

Younes는 현관 앞에 있고 카메라가 그의 사진을 찍습니다 ("arts/camera_0.jpg"). `who_it_is()` 알고리즘이 Younes를 식별하는지 봅시다.

In [ ]:
who_is_it("images/camera_0.jpg", database, FRmodel)

**모범 답안**:

<table>
    <tr>
        <td>
            <b>it's younes, the distance is 0.659393</b>
        </td>
        <td>
           (0.65939283, 'younes')
        </td>
    </tr>
</table>

<br>

"`camera_0.jpg`"(younes 사진)을 "`camera_1.jpg`"(bertrand 사진)로 변경하고 결과를 볼 수 있습니다.

### Congratulations!

- 얼굴 인식 시스템이 잘 작동합니다! 승인 된 사람 만 출입 할 수 있으며, 사람들은 더 이상 ID 카드를 가지고 다닐 필요가 없습니다!
- 이제 최첨단 얼굴 인식 시스템이 어떻게 작동하는지 살펴 보았습니다.

#### 얼굴 인식 모델을 개선하는 방법
여기에서는 구현하지 않겠지만 알고리즘을 더욱 개선 할 수 있는 몇 가지 방법이 있습니다.
- 각 사람의 더 많은 이미지 (다른 조명 조건, 다른 날 촬영 등)를 데이터베이스에 추가합니다. 그런 다음 새 이미지가 주어지면 새 얼굴을 그 사람의 여러 사진과 비교합니다. 이는 정확도를 증가시킬 것입니다.
- 얼굴만 포함하고 얼굴 주변의 "테두리"영역을 줄이도록 이미지를 자릅니다. 이 전처리는 얼굴 주변의 관련없는 픽셀을 제거하고 알고리즘을 더욱 강력하게 만듭니다.

## Key points to remember

- 얼굴 인증은 상대적으로 더 쉬운 1 : 1 매칭 문제를 해결합니다. 얼굴 인식은 더 어려운 1 : K 매칭 문제를 해결합니다.
- 삼중 항 손실(Triplet loss function)은 얼굴 이미지의 인코딩을 학습하기 위해 신경망을 훈련시키는 데 효과적인 손실 함수입니다.
- Verification 및 Recognition에 동일한 인코딩을 사용할 수 있습니다. 두 이미지의 인코딩 사이의 거리를 측정하면 같은 사람의 사진인지 확인할 수 있습니다.

<br>

과제를 마무리하신것을 축하드립니다!

### References:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- The pretrained model we use is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 